In [2]:
import IPython
import numpy as np
import warnings
warnings.filterwarnings("ignore")
from sys import version 
print ('Least-Squares MC for American Options: Conditions for Replication'.center(85,"-"))
print ('Python version:     ' + version )
print ('Numpy version:      ' + np.__version__)
print ('IPython version:    ' + IPython.__version__)
print ('-'*85)

----------Least-Squares MC for American Options: Conditions for Replication----------
Python version:     3.6.5 |Anaconda, Inc.| (default, Mar 29 2018, 13:32:41) [MSC v.1900 64 bit (AMD64)]
Numpy version:      1.14.3
IPython version:    6.4.0
-------------------------------------------------------------------------------------


In [39]:
import numpy as np

class AmericanOptionsLSMC(object):
    """ Class for American options pricing using Longstaff-Schwartz (2001):
    "Valuing American Options by Simulation: A Simple Least-Squares Approach."
    Review of Financial Studies, Vol. 14, 113-147.
    S0 : float : initial stock/index level
    strike : float : strike price
    T : float : time to maturity (in year fractions)
    M : int : grid or granularity for time (in number of total points)
    r : float : constant risk-free short rate
    div :    float : dividend yield
    sigma :  float : volatility factor in diffusion term 
    
    Unitest(doctest): 
    >>> AmericanPUT = AmericanOptionsLSMC('put', 36., 40., 1., 50, 0.06, 0.06, 0.2, 10000  )
    >>> AmericanPUT.price
    4.4731177017712209
    """

    def __init__(self, option_type, S0, strike, T, M, r, div, sigma, simulations):
        try:
            self.option_type = option_type
            assert isinstance(option_type, str)
            self.S0 = float(S0)
            self.strike = float(strike)
            assert T > 0
            self.T = float(T)
            assert M > 0
            self.M = int(M)
            assert r >= 0
            self.r = float(r)
            assert div >= 0
            self.div = float(div)
            assert sigma > 0
            self.sigma = float(sigma)
            assert simulations > 0
            self.simulations = int(simulations)
        except ValueError:
            print('Error passing Options parameters')


        if option_type != 'call' and option_type != 'put':
            raise ValueError("Error: option type not valid. Enter 'call' or 'put'")
        if S0 < 0 or strike < 0 or T <= 0 or r < 0 or div < 0 or sigma < 0:
            raise ValueError('Error: Negative inputs not allowed')

        self.time_unit = self.T / float(self.M)
        self.discount = np.exp(-self.r * self.time_unit)

    @property
    def MCprice_matrix(self, seed = 123):
        """ Returns MC price matrix rows: time columns: price-path simulation """
        np.random.seed(seed)
        MCprice_matrix = np.zeros((self.M + 1, self.simulations), dtype=np.float64)
        MCprice_matrix[0,:] = self.S0
        for t in range(1, self.M + 1):
            brownian = np.random.standard_normal(int(self.simulations / 2))
            brownian = np.concatenate((brownian, -brownian))
            MCprice_matrix[t, :] = (MCprice_matrix[t - 1, :]
                                  * np.exp((self.r - self.sigma ** 2 / 2.) * self.time_unit
                                  + self.sigma * brownian * np.sqrt(self.time_unit)))
        return MCprice_matrix

    @property
    def MCpayoff(self):
        """Returns the inner-value of American Option"""
        if self.option_type == 'call':
            payoff = np.maximum(self.MCprice_matrix - self.strike,
                           np.zeros((self.M + 1, self.simulations),dtype=np.float64))
        else:
            payoff = np.maximum(self.strike - self.MCprice_matrix,
                            np.zeros((self.M + 1, self.simulations),
                            dtype=np.float64))
        return payoff

    @property
    def value_vector(self):
        value_matrix = np.zeros_like(self.MCpayoff)
        value_matrix[-1, :] = self.MCpayoff[-1, :]
        for t in range(self.M - 1, 0 , -1):
            regression = np.polyfit(self.MCprice_matrix[t, :], value_matrix[t + 1, :] * self.discount, 5)
            continuation_value = np.polyval(regression, self.MCprice_matrix[t, :])
            value_matrix[t, :] = np.where(self.MCpayoff[t, :] > continuation_value,
                                          self.MCpayoff[t, :],
                                          value_matrix[t + 1, :] * self.discount)

        return value_matrix[1,:] * self.discount


    @property
    def price(self): return np.sum(self.value_vector) / float(self.simulations)
    
    @property
    def delta(self):
        diff = self.S0 * 0.01
        myCall_1 = AmericanOptionsLSMC(self.option_type, self.S0 + diff, 
                                       self.strike, self.T, self.M, 
                                       self.r, self.div, self.sigma, self.simulations)
        myCall_2 = AmericanOptionsLSMC(self.option_type, self.S0 - diff, 
                                       self.strike, self.T, self.M, 
                                       self.r, self.div, self.sigma, self.simulations)
        return (myCall_1.price - myCall_2.price) / float(2. * diff)
    
    @property
    def gamma(self):
        diff = self.S0 * 0.01
        myCall_1 = AmericanOptionsLSMC(self.option_type, self.S0 + diff, 
                                       self.strike, self.T, self.M, 
                                       self.r, self.div, self.sigma, self.simulations)
        myCall_2 = AmericanOptionsLSMC(self.option_type, self.S0 - diff, 
                                       self.strike, self.T, self.M, 
                                       self.r, self.div, self.sigma, self.simulations)
        return (myCall_1.delta - myCall_2.delta) / float(2. * diff)
    
    @property
    def vega(self):
        diff = self.sigma * 0.01
        myCall_1 = AmericanOptionsLSMC(self.option_type, self.S0, 
                                       self.strike, self.T, self.M, 
                                       self.r, self.div, self.sigma + diff, 
                                       self.simulations)
        myCall_2 = AmericanOptionsLSMC(self.option_type, self.S0,
                                       self.strike, self.T, self.M, 
                                       self.r, self.div, self.sigma - diff, 
                                       self.simulations)
        return (myCall_1.price - myCall_2.price) / float(2. * diff)    
    
    @property
    def rho(self):        
        diff = self.r * 0.01
        if (self.r - diff) < 0:        
            myCall_1 = AmericanOptionsLSMC(self.option_type, self.S0, 
                                       self.strike, self.T, self.M, 
                                       self.r + diff, self.div, self.sigma, 
                                       self.simulations)
            myCall_2 = AmericanOptionsLSMC(self.option_type, self.S0, 
                                       self.strike, self.T, self.M, 
                                       self.r, self.div, self.sigma, 
                                       self.simulations)
            return (myCall_1.price - myCall_2.price) / float(diff)
        else:
            myCall_1 = AmericanOptionsLSMC(self.option_type, self.S0, 
                                       self.strike, self.T, self.M, 
                                       self.r + diff, self.div, self.sigma, 
                                       self.simulations)
            myCall_2 = AmericanOptionsLSMC(self.option_type, self.S0, 
                                       self.strike, self.T, self.M, 
                                       self.r - diff, self.div, self.sigma, 
                                       self.simulations)
            return (myCall_1.price - myCall_2.price) / float(2. * diff)
    
    @property
    def theta(self): 
        diff = 1 / 252.
        myCall_1 = AmericanOptionsLSMC(self.option_type, self.S0, 
                                       self.strike, self.T + diff, self.M, 
                                       self.r, self.div, self.sigma, 
                                       self.simulations)
        myCall_2 = AmericanOptionsLSMC(self.option_type, self.S0, 
                                       self.strike, self.T - diff, self.M, 
                                       self.r, self.div, self.sigma, 
                                       self.simulations)
        return (myCall_2.price - myCall_1.price) / float(2. * diff)

In [40]:
import doctest
doctest.testmod()

**********************************************************************
File "__main__", line ?, in __main__.AmericanOptionsLSMC
Failed example:
    AmericanPUT.price
Expected:
    4.4731177017712209
Got:
    4.473117701771221
**********************************************************************
1 items had failures:
   1 of   2 in __main__.AmericanOptionsLSMC
***Test Failed*** 1 failures.


TestResults(failed=1, attempted=2)

In [41]:
AmericanPUT = AmericanOptionsLSMC('put', 36., 40., 1., 50, 0.06, 0.06, 0.2, 10000  )
print ('Price: ', AmericanPUT.price)
print ('Delta: ', AmericanPUT.delta)
print ('Gamma: ', AmericanPUT.gamma)
print ('Vega:  ', AmericanPUT.vega)
print ('Rho:   ', AmericanPUT.rho)
print ('Theta: ', AmericanPUT.theta)

Price:  4.473117701771221
Delta:  -0.7112251324731934
Gamma:  0.12615233203125087
Vega:   12.196835824506369
Rho:    -10.033522985234042
Theta:  -1.8271728267242384


In [42]:
def prices():
    for S0 in (36., 38., 40., 42., 44.):  # initial stock price values
        for vol in (0.2, 0.4):  # volatility values
            for T in (1.0, 2.0):  # times-to-maturity
                AmericanPUT = AmericanOptionsLSMC('put', S0, 40., T, 50, 0.06, 0.06, vol, 1500)
                print ("Initial price: %4.1f, Sigma: %4.2f, Expire: %2.1f --> Option Value %8.3f" % (S0, vol, T, AmericanPUT.price))

In [44]:
from time import time
t0 = time()
optionValues = prices()  # calculate all values
t1 = time(); d1 = t1 - t0
print ("Duration in Seconds %6.3f" % d1)

Initial price: 36.0, Sigma: 0.20, Expire: 1.0 --> Option Value    4.439
Initial price: 36.0, Sigma: 0.20, Expire: 2.0 --> Option Value    4.779
Initial price: 36.0, Sigma: 0.40, Expire: 1.0 --> Option Value    7.135
Initial price: 36.0, Sigma: 0.40, Expire: 2.0 --> Option Value    8.459
Initial price: 38.0, Sigma: 0.20, Expire: 1.0 --> Option Value    3.225
Initial price: 38.0, Sigma: 0.20, Expire: 2.0 --> Option Value    3.726
Initial price: 38.0, Sigma: 0.40, Expire: 1.0 --> Option Value    6.134
Initial price: 38.0, Sigma: 0.40, Expire: 2.0 --> Option Value    7.666
Initial price: 40.0, Sigma: 0.20, Expire: 1.0 --> Option Value    2.296
Initial price: 40.0, Sigma: 0.20, Expire: 2.0 --> Option Value    2.808
Initial price: 40.0, Sigma: 0.40, Expire: 1.0 --> Option Value    5.201
Initial price: 40.0, Sigma: 0.40, Expire: 2.0 --> Option Value    6.815
Initial price: 42.0, Sigma: 0.20, Expire: 1.0 --> Option Value    1.589
Initial price: 42.0, Sigma: 0.20, Expire: 2.0 --> Option Value  